<a href="https://colab.research.google.com/github/abedmarm/Visualitzacio_de_Dades_PR2/blob/main/main_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a3/Logo_blau_uoc.png/640px-Logo_blau_uoc.png", align="left", width=30%; height=auto>
</div>

<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.959 - Visualització de dades</p>
<p style="margin: 0; text-align:right;">2023-1 · Màster universitari en Ciència de dades (Data science)</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
</div>

# PR2: Projecte de visualització

<div class="alert alert-block alert-info">
<strong>Nom i cognoms: Àlex Bedmar Martínez</strong>
</div>


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.listdir('/content/drive/MyDrive/UOC/(VD) Visualització de dades/PR2')

['GlobalLandTemperaturesByCity.csv',
 'GlobalLandTemperaturesByMajorCity.csv',
 'GlobalTemperatures.csv',
 'merged_df.csv',
 '_map.html',
 'interactive_map.html',
 'temp_map.png',
 'Part II: projecte de visualització.gdoc']

In [4]:
import os
os.chdir('/content/drive/MyDrive/UOC/(VD) Visualització de dades/PR2')

In [5]:
#!pip install cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 24.0 MB/s eta 0:00:00


In [6]:
#!pip install mplcursors

In [7]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import plotly.express as px
import cartopy.crs as ccrs
import ipywidgets as widgets
from IPython.display import display, clear_output
import mplcursors
import matplotlib.colors as mcolors

%matplotlib inline

In [8]:
#data = pd.read_csv('./GlobalLandTemperaturesByMajorCity.csv')
data = pd.read_csv('./GlobalLandTemperaturesByCity.csv')

nRow, nCol = data.shape
print(f'Hi ha {nRow} files i {nCol} columnes')

Hi ha 8599212 files i 7 columnes


In [9]:
df = data.drop(['AverageTemperatureUncertainty', 'Country'], axis=1)

df['dt'] = pd.to_datetime(df['dt'])
df['Year'] = df['dt'].dt.year
df['Month'] = df['dt'].dt.month

df = df.drop(['dt'], axis=1)

def convert_coordinates(coord):
    value, direction = float(coord[:-1]), coord[-1]
    return value if direction in ['N', 'E'] else -value

df['Latitude'] = df['Latitude'].apply(convert_coordinates)
df['Longitude'] = df['Longitude'].apply(convert_coordinates)

df['Month'] = df['Month'].apply(lambda x: pd.Timestamp(2022, x, 1).strftime('%B'))
df['Year'] = df['Year'].astype(int)

df = df.dropna()

# Display the modified DataFrame
print(df)

         AverageTemperature    City  Latitude  Longitude  Year     Month
0                     6.068   Århus     57.05      10.33  1743  November
5                     5.788   Århus     57.05      10.33  1744     April
6                    10.644   Århus     57.05      10.33  1744       May
7                    14.051   Århus     57.05      10.33  1744      June
8                    16.082   Århus     57.05      10.33  1744      July
...                     ...     ...       ...        ...   ...       ...
8599206               7.710  Zwolle     52.24       5.26  2013     April
8599207              11.464  Zwolle     52.24       5.26  2013       May
8599208              15.043  Zwolle     52.24       5.26  2013      June
8599209              18.775  Zwolle     52.24       5.26  2013      July
8599210              18.025  Zwolle     52.24       5.26  2013    August

[8235082 rows x 6 columns]


In [10]:
# Assuming df is your DataFrame
# Convert 'Month' column to categorical with proper ordering
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
df['Month'] = pd.Categorical(df['Month'], categories=month_order, ordered=True)

# Calculate the average temperature for each city and each month
avg_temp_city_month = df.groupby(['City', 'Month'])['AverageTemperature'].mean().reset_index()

# Merge the average temperature back to the original DataFrame
df = pd.merge(df, avg_temp_city_month, on=['City', 'Month'], suffixes=('', '_avg'))

# Calculate the difference in temperature
df['TempDifference'] = df['AverageTemperature'] - df['AverageTemperature_avg']

# Display the resulting DataFrame
print(df)

         AverageTemperature    City  Latitude  Longitude  Year     Month  \
0                     6.068   Århus     57.05      10.33  1743  November   
1                     4.639   Århus     57.05      10.33  1744  November   
2                     5.637   Århus     57.05      10.33  1752  November   
3                     2.871   Århus     57.05      10.33  1753  November   
4                     4.141   Århus     57.05      10.33  1754  November   
...                     ...     ...       ...        ...   ...       ...   
8235077              18.671  Zwolle     52.24       5.26  2009    August   
8235078              16.912  Zwolle     52.24       5.26  2010    August   
8235079              16.975  Zwolle     52.24       5.26  2011    August   
8235080              18.588  Zwolle     52.24       5.26  2012    August   
8235081              18.025  Zwolle     52.24       5.26  2013    August   

         AverageTemperature_avg  TempDifference  
0                      3.928213      

In [11]:
Q1 = df['TempDifference'].quantile(0.25)
Q3 = df['TempDifference'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 6 * IQR
upper_bound = Q3 + 6 * IQR

# Remove outliers
df_filtered = df[(df['TempDifference'] >= lower_bound) & (df['TempDifference'] <= upper_bound)]

# Display the resulting DataFrame without outliers
print(df_filtered)

         AverageTemperature    City  Latitude  Longitude  Year     Month  \
0                     6.068   Århus     57.05      10.33  1743  November   
1                     4.639   Århus     57.05      10.33  1744  November   
2                     5.637   Århus     57.05      10.33  1752  November   
3                     2.871   Århus     57.05      10.33  1753  November   
4                     4.141   Århus     57.05      10.33  1754  November   
...                     ...     ...       ...        ...   ...       ...   
8235077              18.671  Zwolle     52.24       5.26  2009    August   
8235078              16.912  Zwolle     52.24       5.26  2010    August   
8235079              16.975  Zwolle     52.24       5.26  2011    August   
8235080              18.588  Zwolle     52.24       5.26  2012    August   
8235081              18.025  Zwolle     52.24       5.26  2013    August   

         AverageTemperature_avg  TempDifference  
0                      3.928213      

In [12]:
print(df_filtered['TempDifference'].max(), df_filtered['TempDifference'].min())

print(df_filtered['Latitude'].max(), df_filtered['Latitude'].min())

8.994257575757576 -8.998454320987648
69.92 -52.24


In [16]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd

# Map settings
central_lon, central_lat = 0, 35

# Function to update the plot based on user input
def update_plot(central_lon, central_lat, selected_year, selected_month, zoom):
    plt.figure(figsize=(10, 10))
    ax = plt.axes(projection=ccrs.Orthographic(central_lon, central_lat))
    ax.set_global()
    ax.coastlines()

    ax.gridlines(ylocs=range(-90, 91, 10), color='lightgray', linestyle='dashed', linewidth=0.5)

    # Filter DataFrame based on user-selected year and month
    filtered_df = df_filtered[(df_filtered['Year'] == selected_year) & (df_filtered['Month'] == selected_month)]
    month_filtered_df = df_filtered[(df_filtered['Month'] == selected_month)]



    # Set up colormap
    cmap = plt.get_cmap('coolwarm')  # You can choose other colormaps as well

    # Normalize the temperature values to fit into the colormap
    norm = mcolors.Normalize(vmin=month_filtered_df['TempDifference'].min(), vmax=month_filtered_df['TempDifference'].max())

    # Scatter plot of Average Temperature for the selected year and month
    sc = ax.scatter(filtered_df['Longitude'], filtered_df['Latitude'], c=filtered_df['TempDifference'],
                    transform=ccrs.PlateCarree(), s=20, cmap=cmap, norm=norm)

    # Add colorbar
    cbar = plt.colorbar(sc, ax=ax, orientation='horizontal', fraction=0.015, pad=0.05)
    cbar.set_label('Difference in Temperature From City Average')
    cbar.outline.set_visible(False)

    # Zoom in based on user-selected zoom level
    ax.set_extent([central_lon - zoom, central_lon + zoom, central_lat - zoom / 1, central_lat + zoom / 1], crs=ccrs.PlateCarree())

    plt.show()

# Modify DataFrame to order months chronologically and format years as even numbers
df['Month'] = pd.Categorical(df['Month'], categories=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], ordered=True)

# Create interactive widgets with adjusted width using container
lon_slider = widgets.FloatSlider(value=central_lon, min=-180, max=180, step=1, description='West / East')
lat_slider = widgets.FloatSlider(value=central_lat, min=-90, max=90, step=1, description='North / South', orientation='vertical')
year_slider = widgets.IntSlider(value=df['Year'].min(), min=df['Year'].min(), max=df['Year'].max(), step=1,
                                description='Select Year:', style={'description_width': 'initial', 'width': '90%'})
month_dropdown = widgets.Dropdown(options=df['Month'].cat.categories, value=df['Month'].min(), description='Select Month:')
zoom_slider = widgets.FloatSlider(value=65, min=15, max=65, step=1, description='Zoom Level', style={'description_width': 'initial', 'width': '90%'})


# Define an interactive output area
output = widgets.interactive_output(update_plot, {'central_lon': lon_slider, 'central_lat': lat_slider,
                                                  'selected_year': year_slider, 'selected_month': month_dropdown,
                                                  'zoom': zoom_slider})

# Set layout for widgets
widget_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', justify_content='center', padding='10px')
slider_layout = widgets.Layout(display='flex', flex_flow='row', align_items='center', justify_content='flex-start')

lon_slider_output_hbox = widgets.HBox([lon_slider, output], layout=widget_layout)
year_month_zoom_hbox = widgets.HBox([year_slider, month_dropdown, zoom_slider], layout=widget_layout)

# Display widgets and output with different layouts
display(widgets.VBox([lon_slider_output_hbox, lat_slider, year_month_zoom_hbox], layout=slider_layout))

